## Getting started


Check the [quick start tutorial](https://docs.honeycomb.io/quick-start) to learn about Honeycomb UI capabilities. 
Then join the LSST SQuaRE team using [this invitation](https://ui.honeycomb.io/join_team/lsst-square), the datasets created in this notebook are shared with the team.



## What is observability?

The Honeycomb [observability manifesto](https://www.honeycomb.io/blog/2018/03/observability-a-manifesto/) is a good place to start. Also check the introduction to observability [here](https://docs.honeycomb.io/thinking-about-observability/intro-to-observability/).

NOTE: SQuaSH metrics are high level metrics, they are not the kind of 'value' that one would want for observability. However,
in this notebook we'll send them to Honeycomb just because they are the only data that we have in SQuaSH for now. That's still useful to illustrate the use of `libhoney` and `beeline-python` integrations.

In my opinion, the benefit of this tool will become clearer by sending more *context data* from our DM pipeline tasks, and ultimately as an analysis tool for the Engineering and Facilities Database (EFD). 


## Events and Datasets
Honeycomb data is a series of **Events**, each of which represents something in your environment worth tracking. When you send events to Honeycomb, you collect related or recurring events under a single **Dataset**.

An **Event** can be anything:

- an HTTP request to your app
- An SQL query
- A job is submitted to a queue
- An execution of a LSST DM pipeline task
- A detection of particular astronomical object 

**Datasets** are used to partition your data into separate and queryable sets. See [best practices for defining datasets](https://docs.honeycomb.io/getting-data-in/datasets/best-practices/) in Honeycomb. 

Let's start by identifying a `validate_drp` run as the `Event` containing the metrics measured by each run. 

For illustration purposes we'll collect those metrics from the SQuaSH API:

In [1]:
SQUASH_API_URL = "https://squash-restful-api-demo.lsst.codes/"

In [2]:
import requests
r = requests.get(SQUASH_API_URL + "/job/1").json()
    
    
data = {'id': r['id'],
        'date_created': r['date_created'],
        'filter_name': r['meta']['filter_name'],
        'dataset': r['ci_dataset']}
            
# events should be flat dict
for meas in r['measurements']:
    data[meas['metric']] = meas['value']
    
data

{'id': 1,
 'date_created': '2016-09-15T00:00:00Z',
 'filter_name': 'r',
 'dataset': 'validation_data_cfht',
 'validate_drp.AM1': 4.0,
 'validate_drp.AM2': 5.0,
 'validate_drp.PA1': 7.0}

This looks like an structured log entry, it also looks like a property set or it may recall something else...

### Sending events to Honeycomb

1. When honeycomb gets a request to add an event to a dataset that doesn't exist yet, it creates the dataset.
2. The dataset schema is inferred automatically. It also will infer automatically if you add new fields to your events.

3. Events are added to the dataset for querying.

NOTE: In order to execute this cell you will need the HONEY_API_KEY. It is available from "Team Settings" in the UI once you join the `lsst-square` team.

In [3]:
import libhoney

HONEY_API_KEY = ""  # Obtained from the UI under "Team Settings"

libhoney.init(writekey=HONEY_API_KEY, dataset="single-event-demo")
libhoney.send_now(data)

That's it! Check the new dataset at https://ui.honeycomb.io/lsst-square/datasets

NOTE: this demo dataset may already exist from previous executions of this notebook, make sure you delete it before running the notebook if you want to start from an empty dataset. If you don't, note that they will be inserted in the existing dataset.

### Now, let's send all squash data to Honeycomb
Actually, just the scalar metrics and some context information.

Create the `squash-demo` dataset:

In [4]:
libhoney.init(writekey=HONEY_API_KEY, dataset="squash-demo")
builder = libhoney.Builder()

Now loop over all jobs:

In [5]:
from datetime import datetime
    
jobs = requests.get(SQUASH_API_URL + "/jobs").json()

for job_id in jobs['ids']:

    r = requests.get(SQUASH_API_URL + "/job/{}".format(job_id)).json()
    
    if r['ci_dataset'] == 'unknown' or r['ci_dataset'] == 'decam':
        continue
    
    print('Sending event for job {}...'.format(job_id))

    
    # Spawn a new event and override the timestamp
    event = builder.new_event()
    event.add_field('id', job_id)
    event.add_field('filter_name', r['meta']['filter_name'])
    event.add_field('dataset', r['ci_dataset'])
    
    for meas in r['measurements']:
        event.add_field(meas['metric'], meas['value'])
    
    event.created_at = datetime.strptime(r['date_created'], "%Y-%m-%dT%H:%M:%SZ")
    
    event.send()

Sending event for job 1...
Sending event for job 3...
Sending event for job 5...
Sending event for job 7...
Sending event for job 9...
Sending event for job 11...
Sending event for job 13...
Sending event for job 14...
Sending event for job 15...
Sending event for job 16...
Sending event for job 17...
Sending event for job 18...
Sending event for job 19...
Sending event for job 20...
Sending event for job 21...
Sending event for job 22...
Sending event for job 23...
Sending event for job 24...
Sending event for job 25...
Sending event for job 26...
Sending event for job 27...
Sending event for job 28...
Sending event for job 29...
Sending event for job 30...
Sending event for job 31...
Sending event for job 32...
Sending event for job 33...
Sending event for job 34...
Sending event for job 35...
Sending event for job 36...
Sending event for job 37...
Sending event for job 38...
Sending event for job 39...
Sending event for job 40...
Sending event for job 41...
Sending event for job 42.

Sending event for job 293...
Sending event for job 294...
Sending event for job 295...
Sending event for job 296...
Sending event for job 297...
Sending event for job 298...
Sending event for job 299...
Sending event for job 300...
Sending event for job 301...
Sending event for job 302...
Sending event for job 303...
Sending event for job 304...
Sending event for job 305...
Sending event for job 306...
Sending event for job 307...
Sending event for job 308...
Sending event for job 309...
Sending event for job 310...
Sending event for job 311...
Sending event for job 312...
Sending event for job 313...
Sending event for job 314...
Sending event for job 315...
Sending event for job 316...
Sending event for job 317...
Sending event for job 318...
Sending event for job 319...
Sending event for job 320...
Sending event for job 321...
Sending event for job 322...
Sending event for job 323...
Sending event for job 324...
Sending event for job 325...
Sending event for job 326...
Sending event 

Sending event for job 576...
Sending event for job 577...
Sending event for job 578...
Sending event for job 579...
Sending event for job 580...
Sending event for job 581...
Sending event for job 582...
Sending event for job 583...
Sending event for job 584...
Sending event for job 585...
Sending event for job 586...
Sending event for job 587...
Sending event for job 588...
Sending event for job 589...
Sending event for job 590...
Sending event for job 591...
Sending event for job 592...
Sending event for job 593...
Sending event for job 594...
Sending event for job 595...
Sending event for job 596...
Sending event for job 597...
Sending event for job 598...
Sending event for job 599...
Sending event for job 600...
Sending event for job 601...
Sending event for job 602...
Sending event for job 603...
Sending event for job 604...
Sending event for job 605...
Sending event for job 606...
Sending event for job 607...
Sending event for job 608...
Sending event for job 609...
Sending event 

Sending event for job 954...
Sending event for job 955...
Sending event for job 956...
Sending event for job 957...
Sending event for job 958...
Sending event for job 959...
Sending event for job 960...
Sending event for job 961...
Sending event for job 962...
Sending event for job 963...
Sending event for job 964...
Sending event for job 965...
Sending event for job 966...
Sending event for job 967...
Sending event for job 968...
Sending event for job 969...
Sending event for job 970...
Sending event for job 971...
Sending event for job 972...
Sending event for job 973...
Sending event for job 974...
Sending event for job 975...
Sending event for job 976...
Sending event for job 977...
Sending event for job 978...
Sending event for job 979...
Sending event for job 980...
Sending event for job 981...
Sending event for job 982...
Sending event for job 983...
Sending event for job 984...
Sending event for job 985...
Sending event for job 986...
Sending event for job 987...
Sending event 

Sending event for job 1235...
Sending event for job 1236...
Sending event for job 1237...
Sending event for job 1238...
Sending event for job 1239...
Sending event for job 1240...
Sending event for job 1241...
Sending event for job 1242...
Sending event for job 1243...
Sending event for job 1244...
Sending event for job 1245...
Sending event for job 1246...
Sending event for job 1247...
Sending event for job 1248...
Sending event for job 1249...
Sending event for job 1250...
Sending event for job 1251...
Sending event for job 1252...
Sending event for job 1253...
Sending event for job 1254...
Sending event for job 1255...
Sending event for job 1256...
Sending event for job 1257...
Sending event for job 1258...
Sending event for job 1259...
Sending event for job 1260...
Sending event for job 1261...
Sending event for job 1262...
Sending event for job 1263...
Sending event for job 1264...
Sending event for job 1265...
Sending event for job 1266...
Sending event for job 1267...
Sending ev

Sending event for job 1524...
Sending event for job 1525...
Sending event for job 1526...
Sending event for job 1527...
Sending event for job 1528...
Sending event for job 1529...
Sending event for job 1530...
Sending event for job 1531...
Sending event for job 1532...
Sending event for job 1533...
Sending event for job 1534...
Sending event for job 1535...
Sending event for job 1536...
Sending event for job 1537...
Sending event for job 1538...
Sending event for job 1539...
Sending event for job 1540...
Sending event for job 1541...
Sending event for job 1542...
Sending event for job 1543...
Sending event for job 1544...
Sending event for job 1545...
Sending event for job 1546...
Sending event for job 1547...
Sending event for job 1548...
Sending event for job 1549...
Sending event for job 1550...
Sending event for job 1551...
Sending event for job 1552...
Sending event for job 1553...
Sending event for job 1554...
Sending event for job 1555...
Sending event for job 1556...
Sending ev

## Querying the dataset

Querying a Honeycomb dataset in a particular way will produce a series of events (or a time series). The query builder is great tool for fast exploration from the Honeycomb UI. It is also possible to [specify queries programatically](https://docs.honeycomb.io/api/query-specification/).

The SQuaSH `validate_drp.AM1` metric on a given dataset and filter can be obtained from this query:

In [6]:
query = {
    "breakdowns": [
        "dataset", "filter_name"
    ],
    "calculations": [
        {"column": "validate_drp.AM1", "op": "AVG"}
    ],
    "filters":[
      {"column": "dataset", "op": "=", "value": "validation_data_hsc"},
      {"column": "filter_name", "op": "=", "value": "HSC-R"}
    ],
    "filter_combination": "AND"
}

## Markers
Markers are annotations over the time series plot. Markers are defined per dataset and can be created programatically via the [UI, a CLI](https://docs.honeycomb.io/working-with-data/markers/), or the [Markers API](https://docs.honeycomb.io/api/markers/).


In [7]:
headers = {'X-Honeycomb-Team': HONEY_API_KEY}

marker={"message": "Testing marker API"}

r = requests.post("https://api.honeycomb.io/1/markers/squash-demo", json=marker, headers=headers)
r.json()

{'created_at': '2018-09-21T18:11:29Z',
 'updated_at': '2018-09-21T18:11:29Z',
 'start_time': 1537553489,
 'message': 'Testing marker API',
 'id': 'eiEzAM4KH6k'}

All the Markers in a dataset may be retrieved by:

In [8]:
r = requests.get("https://api.honeycomb.io/1/markers/squash-demo", headers=headers)
r.json()

[{'created_at': '2018-09-06T22:57:40Z',
  'updated_at': '2018-09-06T22:57:40Z',
  'start_time': 1533460436,
  'message': 'Another markes',
  'type': 'Custom',
  'id': 'iGm7GHW6hHg'},
 {'created_at': '2018-09-06T22:55:56Z',
  'updated_at': '2018-09-06T22:55:56Z',
  'start_time': 1536274556,
  'message': 'Testing marker API',
  'id': 'GrD43u7xE5H'},
 {'created_at': '2018-09-10T17:27:19Z',
  'updated_at': '2018-09-10T17:27:19Z',
  'start_time': 1536600439,
  'message': 'Testing marker API',
  'id': 'hwNxeBnkbCH'},
 {'created_at': '2018-09-10T20:05:22Z',
  'updated_at': '2018-09-10T20:05:22Z',
  'start_time': 1536609922,
  'message': 'Testing marker API',
  'id': 'v1iwcV1obn'},
 {'created_at': '2018-09-10T20:37:53Z',
  'updated_at': '2018-09-10T20:37:53Z',
  'start_time': 1536611873,
  'message': 'Testing marker API',
  'id': 'ftBSdn81rGv'},
 {'created_at': '2018-09-10T20:47:34Z',
  'updated_at': '2018-09-10T20:47:34Z',
  'start_time': 1536612454,
  'message': 'Testing marker API',
  'id':

## Triggers

Given a time series and a threshold one can use the [Triggers API](https://docs.honeycomb.io/api/triggers/) for setting alerts when the values pass a threshold. Triggers are also defined per dataset. Honeycomb provides integration with Slack for alert notification. 

Let's set an alert for the `validate_drp.AM1` metric. We'll get the design specification for that metric from the SQuaSH API:

In [9]:
spec = requests.get(SQUASH_API_URL + "spec/validate_drp.AM1.design" ).json()
spec

{'name': 'validate_drp.AM1.design',
 'threshold': {'unit': 'marcsec', 'value': 10, 'operator': '<='},
 'tags': ['AM1', 'design', 'achromatic'],
 'metadata_query': {}}

Note that the operator in the `lsst.verify` specifications is such that "measurement `op` spec" is True if the measurement passes the specification. But we need the opposite when configuring alerts. In order to do that we created this mapping: 

In [10]:
inverse_operation = { '>': '<=', '>=': '<', '<': '>=', '<=': '>'}

Also note that triggers do not support equality and inequality, so if converting a `lsst.verify` spec to a Honeycomb trigger and the spec uses an equality match, you will have to decide whether the appropriate trigger comparison is greater than or less than the value.

In [11]:
trigger = {"name": "AM1 alert",
           "query": query,
           "threshold": {"op": inverse_operation[spec["threshold"]["operator"]], 
                         "value": spec["threshold"]["value"]},
           "frequency": 1800
          }
trigger

{'name': 'AM1 alert',
 'query': {'breakdowns': ['dataset', 'filter_name'],
  'calculations': [{'column': 'validate_drp.AM1', 'op': 'AVG'}],
  'filters': [{'column': 'dataset', 'op': '=', 'value': 'validation_data_hsc'},
   {'column': 'filter_name', 'op': '=', 'value': 'HSC-R'}],
  'filter_combination': 'AND'},
 'threshold': {'op': '>', 'value': 10},
 'frequency': 1800}

In [12]:
r = requests.post("https://api.honeycomb.io/1/triggers/squash-demo", json=trigger, headers=headers)
r

<Response [201]>

Finnaly, all Triggers in a dataset may be retrieved by:

In [13]:
r = requests.get("https://api.honeycomb.io/1/triggers/squash-demo", headers=headers)
r.json()

[{'name': 'AM1 alert',
  'frequency': 1800,
  'query': {'breakdowns': ['dataset', 'filter_name'],
   'calculations': [{'column': 'validate_drp.AM1', 'op': 'AVG'}],
   'filters': [{'column': 'dataset',
     'op': '=',
     'value': 'validation_data_hsc'},
    {'column': 'filter_name', 'op': '=', 'value': 'HSC-R'}]},
  'threshold': {'op': '<=', 'value': 10},
  'recipients': [{'type': 'email', 'target': 'alerts@example.com'}],
  'id': 'i4WqTioppHA'},
 {'name': 'AM1 alert',
  'frequency': 1800,
  'query': {'breakdowns': ['dataset', 'filter_name'],
   'calculations': [{'column': 'validate_drp.AM1', 'op': 'AVG'}],
   'filters': [{'column': 'dataset',
     'op': '=',
     'value': 'validation_data_hsc'},
    {'column': 'filter_name', 'op': '=', 'value': 'HSC-R'}]},
  'threshold': {'op': '<=', 'value': 10},
  'recipients': [{'type': 'email', 'target': 'alerts@example.com'}],
  'id': 'wXbfYt59QZE'},
 {'name': 'AM1 alert',
  'frequency': 1800,
  'query': {'breakdowns': ['dataset', 'filter_name']